In [53]:
import sys
import os
import numpy as np
import pandas as pd
from scipy.special import erf
import matplotlib.pyplot as plt
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
from plotly.offline import iplot, init_notebook_mode
plotly .tools.set_credentials_file(username = 'fialhocoelho',\
                                   api_key  = 'XShJogIgb2Yjyd5MpwyT')
sys.path.append('/home/jfialho/git/track-ml/utils')
from tracktop import *



In [54]:
#Sort function

def xyz_swap(df_tb_swap,index_xyz,i,j):
    index_xyz[i], index_xyz[j] = index_xyz[j], index_xyz[i]
    df_tb_swap.iloc[3*i]  , df_tb_swap.iloc[3*j]   = df_tb_swap.iloc[3*j],   df_tb_swap.iloc[3*i]
    df_tb_swap.iloc[3*i+1], df_tb_swap.iloc[3*j+1] = df_tb_swap.iloc[3*j+1], df_tb_swap.iloc[3*i+1]
    df_tb_swap.iloc[3*i+2], df_tb_swap.iloc[3*j+2] = df_tb_swap.iloc[3*j+2], df_tb_swap.iloc[3*i+2]

def xyz_bsort(df_to_be_sorted):
    index_xyz = []
    df_n_col  = df_to_be_sorted.shape[0]//3
    for aux in range(0,df_n_col):
        x = df_to_be_sorted.iloc[3*aux+0]
        y = df_to_be_sorted.iloc[3*aux+1]
        z = df_to_be_sorted.iloc[3*aux+2]
        index_xyz.append(x**2 + y**2 + z**2)
    for i in range(1,len(index_xyz)):
        for j in range(0,len(index_xyz)-i):
            if index_xyz[j]>index_xyz[j+1]:
                xyz_swap(df_to_be_sorted,index_xyz,j,j+1)

In [55]:
default_err      = 0.03
default_err_hard = 0.1
default_err_fad  = 0.174
default_len_err  = 500
default_center   = 1

def err_normal(position, err = default_err, len_err = default_len_err):
    err_array = np.random.normal(loc = default_center, 
                                 scale = err, size=len_err)
    value_err = position * random.choice(err_array)
    return value_err

def err_normal_hard(position, err = default_err_hard, len_err = default_len_err):
    err_array = np.random.normal(loc = default_center, 
                                 scale = err, size=len_err)
    value_err = position * random.choice(err_array)
    return value_err

# PARAMETERS
input_path  = '/data/'
output_path = '/data/track-ml/output/'

# Open the input file as a DF
tracks      = pd.read_csv(input_path + 'real1252.csv')
tracks_real = tracks.iloc[:,0:-1]

tracks_real_sort = tracks_real.copy()

In [56]:
#for i in range(int(tracks_real_sort.shape[0])):
for i in range(int(50)):
    xyz_bsort(tracks_real_sort.iloc[i,:])

In [57]:
# REAL_sorted -> CSV
tracks_real_sort.to_csv(output_path + 'tracks_real_sort.csv', encoding='utf-8', index = False, header = False)

In [58]:
# Aplying error
tracks_fake_sort = tracks_real.iloc[:,:].apply(err_normal)

In [59]:
#FAKE sorted -> CSV
tracks_fake_sort.to_csv(output_path + 'tracks_fake_sort.csv', encoding='utf-8', index = False, header = False)

In [60]:
len_xyz = int(tracks_real_sort.shape[1]/3)

# Initializing lists of indexes
selected_columns_x = np.zeros(len_xyz)
selected_columns_y = np.zeros(len_xyz)
selected_columns_z = np.zeros(len_xyz)

# Generating indexes
for i in range(len_xyz):
    selected_columns_x[i] = int(i*3)
    selected_columns_y[i] = int(i*3+1)
    selected_columns_z[i] = int(i*3+2)
    
# Printing indexes for evaluation 
print('selected_columns of x values from df_hits:\n' + \
      str(selected_columns_x) + \
      '\n\nselected_columns of y values from df_hits:\n' + \
      str(selected_columns_y) + \
      '\n\nselected_columns of z values from df_hits:\n' + \
      str(selected_columns_z) + \
      '\n\n len of selected_columns x,y,z: ' + \
      str(len(selected_columns_x)) + ', ' +
      str(len(selected_columns_x)) + ', ' +
      str(len(selected_columns_x)))

selected_columns of x values from df_hits:
[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27. 30. 33. 36. 39. 42. 45. 48. 51.
 54. 57.]

selected_columns of y values from df_hits:
[ 1.  4.  7. 10. 13. 16. 19. 22. 25. 28. 31. 34. 37. 40. 43. 46. 49. 52.
 55. 58.]

selected_columns of z values from df_hits:
[ 2.  5.  8. 11. 14. 17. 20. 23. 26. 29. 32. 35. 38. 41. 44. 47. 50. 53.
 56. 59.]

 len of selected_columns x,y,z: 20, 20, 20


In [65]:
# number of tracks
n_track = 10

# To plot All the tracks
# n_track = df_hits.shape[0]
# !!!It Will be very time consuming !!!

color_real      = 'blue'
color_fake      = 'red'
plot_name       = 'plot_track_real_fake'
plot_extension  = '.webp'

# list os data to plot
data = []
track = [None] * n_track

# loop to create the color_fake traces of fake tracks (df_is_real.iloc[i] == 1.0),
# or/and the color_real traces of real tracks (df_is_real.iloc[i] == 0.0)

#real

for i in range(n_track):
    track[i] = go.Scatter3d(
        # Removing null values (zeroes) in the plot
        x = tracks_real_sort.replace(0.0, np.nan).iloc[i,selected_columns_x],
        y = tracks_real_sort.replace(0.0, np.nan).iloc[i,selected_columns_y],
        z = tracks_real_sort.replace(0.0, np.nan).iloc[i,selected_columns_z],
        # x,y,z data with null values (zeroes)
        #
        # x = df_hits_x.iloc[i,:],
        # y = df_hits_y.iloc[i,:],
        # z = df_hits_z.iloc[i,:],
        marker = dict(
            size = 1,
            color = color_real,
        ),
        line = dict(
            color = color_real,
            width = 1
        )
    )
    # append the track[i] in the list for plotting
    data.append(track[i])


#real without sort
'''
for i in range(n_track):
    track[i] = go.Scatter3d(
        # Removing null values (zeroes) in the plot
        x = tracks_real.replace(0.0, np.nan).iloc[i,selected_columns_x],
        y = tracks_real.replace(0.0, np.nan).iloc[i,selected_columns_y],
        z = tracks_real.replace(0.0, np.nan).iloc[i,selected_columns_z],
        # x,y,z data with null values (zeroes)
        #
        # x = df_hits_x.iloc[i,:],
        # y = df_hits_y.iloc[i,:],
        # z = df_hits_z.iloc[i,:],
        marker = dict(
            size = 1,
            color = color_fake,
        ),
        line = dict(
            color = color_fake,
            width = 1
        )
    )
    # append the track[i] in the list for plotting
    data.append(track[i])
'''

#fake
'''
for i in range(n_track):
    track[i] = go.Scatter3d(
        # Removing null values (zeroes) in the plot
        x = tracks_fake_sort.replace(0.0, np.nan).iloc[i,selected_columns_x],
        y = tracks_fake_sort.replace(0.0, np.nan).iloc[i,selected_columns_y],
        z = tracks_fake_sort.replace(0.0, np.nan).iloc[i,selected_columns_z],
        # x,y,z data with null values (zeroes)
        #
        # x = df_hits_x.iloc[i,:],
        # y = df_hits_y.iloc[i,:],
        # z = df_hits_z.iloc[i,:],
        marker = dict(
            size = 1,
            color = color_fake,
        ),
        line = dict(
            color = color_fake,
            width = 1
        )
    )
    # append the track[i] in the list for plotting
    data.append(track[i])
'''

layout = dict(
    width    = 900,
    height   = 750,
    autosize = False,
    title    ='Tracking real x fake',
    scene = dict(
        xaxis = dict(
            gridcolor       = 'rgb(255, 255, 255)',
            zerolinecolor   = 'rgb(255, 255, 255)',
            showbackground  = True,
            backgroundcolor = 'rgb(230, 230,230)',
            title           ='x (mm)'
        ),
        yaxis=dict(
            gridcolor       = 'rgb(255, 255, 255)',
            zerolinecolor   = 'rgb(255, 255, 255)',
            showbackground  = True,
            backgroundcolor = 'rgb(230, 230,230)',
            title           = 'y (mm)'
        ),
        zaxis=dict(
            gridcolor       = 'rgb(255, 255, 255)',
            zerolinecolor   = 'rgb(255, 255, 255)',
            showbackground  = True,
            backgroundcolor = 'rgb(230, 230,230)',
            title           = 'z (mm)'
        ),
        camera = dict(
            up = dict(
                x = 0,
                y = 0,
                z = 1
            ),
            eye = dict(
                x = -1.7428,
                y = 1.0707,
                z = 0.7100,
            )
        ),
        aspectratio = dict( x = 1, y = 1, z = 0.7),
        aspectmode = 'manual'
    ),
)

fig = dict(data = data, layout = layout)

py.iplot(fig, filename='plot_track_real_fake', height=700)